# 動吸振器

吉田勝俊（宇都宮大学）

## 参考情報
- [Welcome to SymPy’s documentation! &#8212; SymPy 1.10.1 documentation](https://docs.sympy.org/) （本家）
- [SymPy による数式処理とグラフ作成 - 弘前大学 Home Sweet Home](https://home.hirosaki-u.ac.jp/jupyter/sympy/)
- [Pythonで運動方程式を解く(odeint) - Qiita](https://qiita.com/binaryneutronstar/items/ad5efa27fd626826846f)
- [[Python] Numpyの参照、抽出、結合 - Qiita](https://qiita.com/supersaiakujin/items/d63c73bb7b5aac43898a)
- [【Python】行列指数関数・行列対数関数 - Qiita](https://qiita.com/Mrrmm252/items/a50a9b352e5064e40cc1)

In [ ]:
%matplotlib inline

import sympy as sym #数式処理ライブラリ
import numpy as np                 #数値計算ライブラリ
from scipy.integrate import odeint #常微分方程式ライブラリ
import scipy.linalg as la          #線形代数ライブラリ
import matplotlib.pyplot as plt    #描画ライブラリ
plt_config = {
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 14,
    'lines.linewidth': 1.2,
    'lines.markersize': 3,
    'lines.markeredgewidth': 0.7,
    'lines.markerfacecolor': 'white',
    'lines.markeredgecolor': 'black',
}
plt.rcParams.update(plt_config)

In [ ]:
def display_results(result_dic, keys):
    '''
    計算結果の内容の表示する
    '''
    for key in keys:
        print(key, '= ')
        value = result_dic[key]
        
        if isinstance(value, list):
            display(sym.Matrix(value).T)
        else:
            display(result_dic[key])
            
        print('-----')

## ◯状態方程式の導出

### ■ユーザー関数

In [ ]:
def Derive_ELE(system):
    '''
    オイラー・ラグランジュ方程式（ELE: Euler-Lagrange equation）
    「左辺＝０」の左辺を導出する関数
    
    def system():    
        t: 時間（スカラ）
        q: 一般化座標（ベクトル）
        L: ラグランジュ関数（スカラ）
        D: 散逸関数（スカラ）
        Q: 一般化力（ベクトル）
        u: 外部入力（ベクトル）※線形化で用いる
        plist: パラメータ（リスト）
        
        return {'t':t, 'q':q, 'L':L, 'D':D, 'Q':Q, 'u':u, 'plist':plist} #辞書型
    '''
    t, q, L, D, Q, u = [ #plistは未使用
        system()[key]
        for key in ['t','q','L','D','Q','u']
    ]
    dqdt             = sym.diff(q, t) #一般化座標の時間微分
    dim              = len(q)         #一般化座標の次元
    
    ### ラグランジュの運動方程式 ###
    EOM = sym.Matrix([
        sym.simplify(
            sym.diff( sym.diff(L, dqdt[i]), t )
            - sym.diff(L, q[i])
            + sym.diff(D, dqdt[i])
            - (Q[i] if Q is not None else 0) #Q が無ければ 0
        )
        for i in range(dim)
    ])
    
    ELE = {'EOM':EOM}
    ELE.update(system()) #system()の結果を継承

    return ELE

In [ ]:
def FirstOrdered(ELE):
    '''
    運動方程式を1階化するユーザ関数
    - ELE := Derive_ELE(system)の出力
    '''
    EOMq, q, t, u = [
        ELE[key]
        for key in ['EOM','q','t','u']
    ]

    ### 状態ベクトルの生成 ###
    # 実体
    xq = sym.Matrix(
        [qi for qi in q] 
        + #リストの + は連結
        [sym.diff(qi, t) for qi in q]
    )

    # 通し番号
    x = sym.Matrix([
        sym.Function(r'x_' + str(i+1), real=True)(t)
        for i in range(len(xq))
    ])

    # 加速度（通し番号）
    accel = sym.Matrix([
        sym.diff(xi, t)
        for xi in x
    ])

    ### 状態ベクトルの生成 ###
    dim = len(q)
    EOMx = []
    for eqn in EOMq:
        
        #運動方程式を通し番号の状態ベクトルで表す
        for i in range(dim):
            eqn = eqn.subs(xq[dim+i], x[dim+i])
            eqn = eqn.subs(xq[i], x[i])
            
        EOMx.append(eqn)

    ### 加速度について解く ###
    sols = sym.solve(EOMx, accel[dim:])
    
    ### 1階化した運動方程式 dx/dt = f(x) の右辺 ###
    fx = sym.Matrix(
        x[dim:] 
        + [sols[a].simplify() for a in accel[dim:]]
    )

    first_ordered = {'fx':fx, 'x':x, 'xq':xq}
    first_ordered.update(ELE) #ELEを継承
    
    return first_ordered

In [ ]:
def initial_state(x0, x0symb, x):
    '''
    線形化の基準点が未指定なら記号で生成する
    '''
    if x0 is None:
        if len(x) == 1:  #基準点がスカラ（1次元）の場合
            x0 = sym.Matrix([x0symb])
        else:            #基準点がベクトル（多次元）の場合
            x0 = sym.Matrix([
                sym.symbols(x0symb + '_' + str(i+1)) for i in range(len(x))
            ])

    else:
        x0 = sym.Matrix(x0)
    return x0

In [ ]:
def Linearized(first_ordered, x0=None, u0=None):
    '''
    1階化した運動方程式を，x0 の近傍で線形化する
    - first_ordered ... FirstOrdered(ELE) の出力
    '''
    fx, x, xq, t, u = [
        first_ordered[key]
        for key in ['fx','x','xq','t','u']
    ]

    # 状態ベクトルに関するヤコビ行列
    dfxdx = fx.jacobian(x)          #ヤコビ行列
    x0 = initial_state(x0, 'x0', x) #基準点の準備
    for xi, x0i, in zip(x, x0):     #基準点の代入（状態）
        dfxdx = dfxdx.subs(xi, x0i)

    dfxdx = sym.simplify(dfxdx)
        
    jacobi_x = {'A':dfxdx, 'x0':x0} #(ヤコビ行列，基準点)
    
    # 外部入力に関するヤコビ行列
    if u is not None: #もし外部入力が存在すれば
        dfxdu = fx.jacobian(u)          #ヤコビ行列
        u0 = initial_state(u0, 'u0', u) #基準点の準備
        for ui, u0i, in zip(u, u0):     #基準点の代入（入力）
            dfxdu = dfxdu.subs(ui, u0i)
        for xi, x0i, in zip(x, x0):     #基準点の代入（状態）
            dfxdu = dfxdu.subs(xi, x0i)
            
        dfxdu = sym.simplify(dfxdu)
        jacobi_u = {'B':dfxdu, 'u0':u0} #(ヤコビ行列，基準点)
        
    else:
        jacobi_u = {'B':None, 'u0':None}

    ### 結果（辞書型） ###
    linearized = {}
    linearized.update(jacobi_u)      #xの(ヤコビ行列，基準点)をマージ
    linearized.update(jacobi_x)      #uの(ヤコビ行列，基準点)をマージ
    linearized.update(first_ordered) #継承
        
    return linearized

### ■動吸振器（DVA: dynamic vibration absorber）

- 一般化座標: $(q_1,q_2)$

In [ ]:
def system_DVA():
    '''
    システムを定義する関数（動吸振器）
    '''
    # パラメータ: 
    t, m, c1, k1, a, c2, k2 = sym.symbols(
        r't m c1 k1 a c2 k2', 
        positive=True #正の実数に制限 ※なるべく制限すると simplify がよく効く．以下同．
    )
    plist = [m, c1, k1, a, c2, k2] #パラメータのリスト
    
    # 一般化座標（時間関数）: 
    q = sym.Matrix([
        sym.Function(r'q1', real=True)(t),
        sym.Function(r'q2', real=True)(t),
    ])
        
    dq = sym.diff(q, t) #その時間微分
    
    # 運動エネルギー
    T = (m/2)*(dq[0])**2 + (a*m/2)*(dq[1])**2
    
    # 位置エネルギー
    U = (k1/2)*(q[0])**2  + (k2/2)*(q[1]-q[0])**2
    
    # ラグランジュ関数
    L = T - U

    # 散逸関数
    D = (c1/2)*(dq[0])**2 + (c2/2)*(dq[1]-dq[0])**2

    # 外部入力
    u = sym.Matrix([ sym.Function(r'u', real=True)(t) ])
    
    # 一般化力（主系の質点のみに外力が作用）
    Q = sym.Matrix([
        u[0], 
        0 
    ])

    return {'t':t, 'q':q, 'L':L, 'D':D, 'Q':Q, 'u':u, 'plist':plist}

#### 運動方程式

In [ ]:
ele_DVA = Derive_ELE(system_DVA)

display_results(ele_DVA, ['EOM', 'q'])

#### 1階化

In [ ]:
first_DVA = FirstOrdered(ele_DVA)

display_results(first_DVA, ['fx','x','xq'])

#### ☆線形状態方程式の導出結果（LSE: linear state equation）

- 行列を取り出すのに，`Linearized()` が流用できる

In [ ]:
lse_DVA = Linearized(first_DVA)

display_results(lse_DVA, ['xq', 'A', 'x0', 'B', 'u0'])

## ◯伝達関数行列

In [ ]:
def TransFuncMat(linearized):
    '''
    線形状態方程式から伝達関数行列を計算する
    - linearized ... Linearized(first_ordered)の出力
    '''
    A = linearized['A']
    B = linearized['B']
    nrow, ncol = A.shape
    
    s = sym.symbols('s')
    E = sym.eye(nrow)
    
    G = (s*E - A)**(-1)*B
    G = sym.simplify(G)
    
    trans_func_mat = {'G':G, 's':s}
    trans_func_mat.update(linearized) #継承
    
    return trans_func_mat

In [ ]:
Gs_DVA = TransFuncMat(lse_DVA) #計算に時間がかかります

display_results(Gs_DVA, ['G','s','plist'])

#### Numpy 関数への変換

In [ ]:
def numpyTransFuncMat(trans_func_mat):
    '''
    sympy の伝達関数行列を，numpy 関数に変換する
    '''
    G = trans_func_mat['G']
    s = trans_func_mat['s']
    plist = trans_func_mat['plist']
    print(plist)
    
    numpy_func = sym.lambdify([s]+plist, G, modules='numpy')
    
    return numpy_func

In [ ]:
numGs_DVA = numpyTransFuncMat(Gs_DVA)

動作確認

In [ ]:
print( numGs_DVA(s=1j, m=2, c1=3, k1=4, a=5, c2=6, k2=7).T )
print( numGs_DVA(1j, 2, 3, 4, 5, 6, 7).T )

## ◯周波数応答

In [ ]:
def FreqRes_DVA(Gfunc, params):
    '''
    動吸振器の周波数応答を求める．
    '''
    om = np.linspace(0.1, 2, 500) #周波数軸を表す等差数列
    
    Gjw = Gfunc(om*1j, *params)
    
    Gjw_mod = np.array([
        Gjw[0,0,:],             #主系の変位
        Gjw[1,0,:] - Gjw[0,0,:] #従系の相対変位
    ])
    
    R = np.abs(Gjw_mod) #振幅比
    phi = np.angle(Gjw_mod)
    
    return (om, R, phi)

In [ ]:
freq_res_DVA = FreqRes_DVA(numGs_DVA, [1, 0.01, 1, 0.05, 0.0127, 0.0454])

In [ ]:
def plot_FreqRes(freq_res):
    
    oms, Rs, phis = freq_res

    fig, ax = plt.subplots(2,1, figsize=(5,5))
    

    ### 主系
    ax[0].plot(oms, Rs[0,:],   '-k', label=r'$R_1(\omega)$')
    ax[1].plot(oms, phis[0,:], '-k', label=r'$\phi_1(\omega)$')

    ### 従系
    ax[0].plot(oms, Rs[1,:], ':k', label=r'$R_{2-1}(\omega)$')
    # 位相差を<0に補正
    phis1 = np.array([
        phi if phi < 0 else phi - 2*np.pi 
        for phi in phis[1,:]
    ])
    ax[1].plot(oms, phis1,   ':k', label=r'$\phi_{2-1}(\omega)$')

    ax[0].set_ylabel(r'$R(\omega)$')
    ax[1].set_ylabel(r'$\phi(\omega)$')
    ax[1].set_xlabel(r'$\omega$')
    for a in ax:
        a.legend()

In [ ]:
freq_res_DVA_untuned = FreqRes_DVA(
    numGs_DVA,
    [1, 0.01, 1, 0.05, 0.01, 100]
)
plot_FreqRes(freq_res_DVA_untuned)

#plt.savefig('Code_9.1_DVA_untuned.pdf')

In [ ]:
freq_res_DVA_tuned = FreqRes_DVA(
    numGs_DVA,
    [1, 0.01, 1, 0.05, 0.0127, 0.0454]
)
plot_FreqRes(freq_res_DVA_tuned)

#plt.savefig('Code_9.1_DVA_tuned.pdf')